1
<img scr='./module12_files/m8_b2_auth.JPG'>

In [ ]:
./module12_files/m8_b2_auth.JPG

# Итак, давайте пройдем процедуру получения отладочного токена:
# Шаг 1. 
Заходим на https://oauth.yandex.ru/ и нажимаем кнопку "Зарегистрировать новое приложение"

На открывшейся странице даем приложению название (можно выбрать любое, в нашем случае название мало на что влияет):

Далее в пункте "Платформы" выберите вариант "Веб-сервисы" и нажмите "Подставить URL для разработки". Это позволит нам сгенерировать отладочный токен в браузере:

Пролистываем страницу вниз до пункта "Доступы".

Здесь необходимо выбрать к каким сервисам ваше приложение будет иметь доступ. Например, если вам необходимо работать с Яндекс.Вебмастером, то без выбора нужных пунктов в меню "Яндекс.Вебмастер" ваше приложение никогда не получит от API требуемых данных:

Находим в списке Яндекс.Метрику и выбираем пункт получения статистики:

Этот набор прав позволяет получать любые отчеты по счетчикам, которые доступны вам в веб-интерфейсе под данной учетной записью.

Второй пункт "Создание счётчиков..." подразумевает расширенный доступ к настройке счетчиков: добавление пользователей, создание целей и другие возможности. Такие действия при некорректном использовании могут привести к необратимой потере данных. Поэтому для первого знакомства с API лучше использовать только первый пункт "Получение статистики".

# Шаг 2. 
Отматываем страницу в самый низ и нажимаем кнопку "Создать приложение". После успешного создания приложения вы должны увидеть экран:

Проверьте, что в поле "Callback URL" у вас стоит URL для разработки. Если его нет, значит в пункте "Платформа" вы не подставили в свойства своего приложения URL для разработки.

# Шаг 3. Осталось получить отладочный токен.

Процедура детально описана на странице https://tech.yandex.ru/oauth/doc/dg/tasks/get-oauth-token-docpage/ и сводится к следующим действиям:

Подставьте в адрес https://oauth.yandex.ru/authorize?response_type=token&client_id= в конец значение ID вашего приложения. На скриншоте выше ID приложения указывается в поле ID (первый серый прямоугольник).

В открывшейся странице разрешите доступ приложения к вашим данным в Яндекс.Метрике:

На экране появится токен, который вы будете использовать в запросах к Яндекс.Метрике. Сохраните его для дальнейшего использования (например, в файл).

Основы работы с Яндекс.Метрикой
У Яндекса есть небольшой онлайн-курс по работе с Яндекс.Метрикой. Ознакомиться с ним вы можете по ссылке.
https://yandex.ru/adv/edu/metrika-start

In [2]:
import yaml

In [6]:
with open('./module12_files/yandex_config.yaml', 'r', encoding='utf-8') as f:
    token = yaml.load(f)['access_token']

# Обработка ошибок API
Далее несколько рекомендации по наиболее эффективной и безопасной работе со счетчиками.

В начале работы с API неизбежны ошибки. Перед тем как начинать отправлять запросы к Метрике давайте разберем как интерпретировать ошибки. Например, чтобы различать случаи ошибки в названии метрики, неправильном номере счетчика или недоступности сервиса.

Мы изучаем это раньше, чем код запроса, чтобы вы могли в каждый момент практических упражнений понимать, с какими ошибками столкнулись. 

Отправим для примера запрос, допустив ошибку в номере счетчика. Выполним код с соответствующей ошибкой. 

Номер счетчика задается в параметре id, мы сделали его заведомо "неверным":

In [7]:
import requests

API_URL = 'https://api-metrika.yandex.ru/stat/v1/data'

params = {

    'date1': '2018-01-01',

    'date2': '2018-01-01',

    'preset': 'traffic',

    'id': '12345'

}

requests.get(API_URL, params = params).json()

{'errors': [{'error_type': 'not_found', 'message': 'Entity not found'}],
 'code': 404,
 'message': 'Entity not found'}

В данном примере в качестве ответа мы получили сообщение об ошибке. Оно содержит код ошибки 404 и сообщение 'message'. Смысл ошибки в этом случае понятен из сообщения (счетчика с таким номером не существует), но можно уточнить на странице https://tech.yandex.ru/metrika/doc/api2/api_v1/concept/errors-docpage/. Рассмотрим часто встречающиеся ошибки при запросах к API:

access_denied (403) - ошибка доступа к данным. Скорее всего вы забыли подставить токен или неправильно указали номер счетчика (т. е. запрос идет к данным чужого счетчика).

missing_parameter (400) - в запросе не указан необходимый параметр

Код ошибки 429 - вы отправили слишком много запросов сегодня. Обычно лимиты на использование API Метрики обновляются раз в день, т. е. надо просто подождать. Подробнее о лимитах можно узнать на странице https://tech.yandex.ru/metrika/doc/api2/intro/quotas-docpage/

query_error (400) - слишком сложный запрос. Надо уменьшить точность отчета (параметр accuracy) или разбить на несколько запросов по меньшим периодам (если это возможно для данного отчета)

backend_error (503) - самая плохая ошибка. Что-то сломалось в Метрике и сервис недоступен. Остается ждать или написать в службу техподдержки